<a href="https://colab.research.google.com/github/MateoRivera/Deep-Natural-Language-Processing-Laboratories/blob/main/LAB1/Practice_1_(Solution%20BY%20MATEO%20RIVERA)_Text_processing_and_topic_modeling%20-%20copia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Deep Natural Language Processing @ PoliTO**

---


**Teaching Assistant:** Giuseppe Gallipoli

**Credits:** Moreno La Quatra

**Solution by:** Mateo Rivera Monsalve

**Practice 1:** Text processing and topic modeling

# **Text processing**
---
The text processing phase is a preliminary stage where the text to be manipulated is processed to be ready for subsequent analysis.

Text processing usually entails several steps that could possibly include:
- **Language Identification**: identifying the language of a given text.
- **Tokenization**: splitting a given text in several sentences/words.
- **Dependency tree parsing:** analyzing the depencies between words composing the text.
- **Stemming/Lemmatization:** obtain the root form for each word in text.
- **Stopword removal**: removing words that are si commonly used that they carry very little useful information.
- **Part of Speech Tagging:** given a word, retrieve its part of speech (proper noun, common noun or verb).



### Language Identification

| Text                                                                                                                                | Language Code |
|-------------------------------------------------------------------------------------------------------------------------------------|---------------|
| The "Deep Natural Language Processing" course is offered during the first semester of the second year at Politecnico di Torino      | `EN`            |
| Il corso "Deep Natural Language Processing" viene impartito al Politecnico di Torino durante il primo semestre del secondo anno.    | `IT`            |
| Le cours "Deep Natural Language Processing" est enseigné au Politecnico di Torino pendant le premier semestre de la deuxième année. | `FR`            |

**Language Identification** is a crucial prelimiary step because each language has its own characteristics. The knowledge of the main language associated to a given text could be beneficial for all subsequent steps in text processing pipeline.

The data collection used in this first part of the practice is provided [here](https://github.com/MorenoLaQuatra/DeepNLP/blob/main/practices/P1/langid_dataset.csv) - [source: Kaggle](https://www.kaggle.com/martinkk5575/language-detection)


# Exercise 1:

Benchmark different language-detection algorithm by computing the accuracy of each approach:
- [fastlangid](https://pypi.org/project/fastlangid/) (built on FastText)
- [LangID](https://github.com/saffsd/langid.py)
- [langdetect](https://pypi.org/project/langdetect/)

**Hint:** language code conversion: [iso639-lang](https://pypi.org/project/iso639-lang/)

For each method report:
- Accuracy
- Average time per example

In [2]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/langid_dataset.csv

--2025-02-25 15:19:02--  https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/langid_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12990065 (12M) [application/octet-stream]
Saving to: ‘langid_dataset.csv.1’

langid_dataset.csv. 100%[===================>]  12.39M  58.7MB/s    in 0.2s    

2025-02-25 15:19:02 (58.7 MB/s) - ‘langid_dataset.csv.1’ saved [12990065/12990065]



The first step will be installing the different libraries

In [3]:
!pip install fastlangid
!pip install langid
!pip install langdetect
!pip install iso639-lang

In [4]:
# Handle the dataset
import pandas as pd
import time

# Handle text processing
from iso639 import Lang
from fastlangid.langid import LID
from langdetect import detect
import langid

In [5]:
path_langid_dataset = 'langid_dataset.csv'
langid_df = pd.read_csv(path_langid_dataset)
langid_df.head()

,Text,language
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian
1,sebes joseph pereira thomas på eng the jesuit...,Swedish
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil
4,de spons behoort tot het geslacht haliclona en...,Dutch


Let's extract the language code iso639 for each language

In [6]:
langid_df['language_code'] = langid_df['language'].apply(lambda l: Lang(l).pt1)
langid_df.head()

,Text,language,language_code
0,klement gottwaldi surnukeha palsameeriti ning ...,Estonian,et
1,sebes joseph pereira thomas på eng the jesuit...,Swedish,sv
2,ถนนเจริญกรุง อักษรโรมัน thanon charoen krung เ...,Thai,th
3,விசாகப்பட்டினம் தமிழ்ச்சங்கத்தை இந்துப் பத்திர...,Tamil,ta
4,de spons behoort tot het geslacht haliclona en...,Dutch,nl


In [7]:
def accuracy(df, y, y_pred):
    return sum(df[y] == df[y_pred]) / len(df)

## fastlangid

In [8]:
fastlangid = LID()

start = time.time()
langid_df['fastlangid_prediction'] = langid_df['Text'].apply(fastlangid.predict)
fastlangid_time_avg = (time.time() - start) / len(langid_df)

print(f"Accuracy: {accuracy(langid_df, 'language_code', 'fastlangid_prediction')}")
print(f"Average time per example: {fastlangid_time_avg}")

Accuracy: 0.9223636363636364
Average time per example: 0.0005043091340498491


## langid

In [9]:
start = time.time()
langid_df['langid_prediction'] = langid_df['Text'].apply(lambda text: langid.classify(text)[0])
langid_time_avg = (time.time() - start) / len(langid_df)

print(f"Accuracy: {accuracy(langid_df, 'language_code', 'langid_prediction')}")
print(f"Average time per example: {langid_time_avg}")

Accuracy: 0.9542727272727273
Average time per example: 0.003195858294313604


## langdetect
Initially, I executed the following script:
```
langid_df['langdetect_prediction'] = langid_df['Text'].apply(detect)
```
However, I encountered an error: `LangDetectException: No features in text.` This issue was resolved with the help of a [Stack Overflow article](https://stackoverflow.com/questions/40783383/error-using-langdetect-in-python-no-features-in-text).

To address this problem, I defined a custom function `langdetect_custom` which includes a try-except statement. In case the code generates an exception, it will return `None`



In [10]:
for i, text in enumerate(langid_df['Text']):
    try:
        detect(text)
    except:
        print(f"{i}-th text", text)

7381-th text ﺩﺍﻭﺩﺍﺳﻪ ﻓﻀﯿﻠﺖ ﭘﻪ ﺍﺣﺎﺩﯾﺜﻮﮐﻲ  – ﺣﻤﺮﺍﻥ ﺭﻭﺍﯾﺖ ﮐﻮﯼ ﭼﯥ ﯾﻮﻩ ﻭﺭځ ﺣﻀﺮﺕ ﻋﺜﻤﺎﻥ ﺑﻦ ﻋﻔﺎﻥ ﺭﺿﯽ ﺍﻟﻠﻪ ﻋﻨﻪ ﭘﻪ ښﻪ ډﻭﻝ ﺳﺮﻩ ﺍﻭﺩﺱ ﺗﺎﺯﻩ ﮐړ ﺍﻭﺑﯿﺎﯾﯽ ﻭﻓﺮﻣﺎﯾﻞ ﻣﺎﺭﺳﻮﻝ ﺍﻟﻠﻪ ﺻﻠﯽ ﺍﻟﻠﻪ ﻋﻠﯿﻪ ﻭﺳﻠﻢ ﭘﺮﺍﻭﺩﺍﺳﻪ ﻭﻟﯿﺪﯼ ﭘﻪ ښﻪ ډﻭﻝ ﺳﺮﻩ ﺋﯥ ﺍﻭﺩﺱ ﺗﺎﺯﻩ ﮐړ ﺍﻭﺑﯿﺎﯾﯽ ﻭﻓﺮﻣﺎﯾﻞ
18176-th text                                          
19361-th text  – ﺩﺣﻀﺮﺕ ﺍﺑﻮﻫﺮﯾﺮﻩ ﺭﺿﯽ ﺍﻟﻠﻪ ﻋﻨﻪ څﺨﻪ ﺭﻭﺍﯾﺖ ﺩﯼ ﭼﯥ ﻣﺎﺩﻧﺒﯽ ﮐﺮﯾﻢ ﺻﻠﯽ ﺍﻟﻠﻪ ﻋﻠﯿﻪ ﻭﺳﻠﻢ ﻧﻪ ﻭﺍﻭﺭﯾﺪﻝ ﭼﯥ ﺩﺍﺍﻣﺖ ﺑﻪ ﺩﻗﯿﺎﻣﺖ ﭘﻪ ﻭﺭځ ﺑﺎﻧﺪﯼ ﺭﺍﻭﻏﻮښﺘﻞ ﺷﻲ ﭼﯥ ﺩﺍﻭﺩﺍﺳﻪ ﻟﻪ ﮐﺒﻠﻪ ﺑﻪ ﺩﺩﻭﯼ ﻻﺳﻮﻧﻪ ﭘښﯥ ﺍﻭﻣﺨﻮﻧﻪ ﻧﻮﺭﺍﻧﻲ ﺍﻭﺭﻭښﺎﻧﻪ ﻭﻱ څﻮﮎ ﭼﯥ ﺧﭙﻠﻪ ﺭﻭښﻨﺎﯾﯽ ﺯﯾﺎﺗﻮﯼ ﻧﻮﺯﯾﺎﺗﯽ ﺩﯼ ﮐړﻱﺭﻭﺍﻩ ﺑﺨﺎﺭﻱ ﺍﻟﺘﺮﻏﯿﺐ ﻭﺍﻟﺘﺮﻫﯿﺐ ﻟﻮﻣړﯼ ټﻮﮎ  ﭘﺎڼﻪ ﺣﺪﯾﺚ   ﻟﯿﮑﻮﺍﻝ ﺣﺎﻓﻆ ﺯﮐﻲ ﺍﻟﺪﯾﻦ ﻋﺒﺪﺍﻟﻌﻈﯿﻢ ﺑﻦ ﻋﺒﺪﺍﻟﻘﻮﻱ ﺍﻟﻤﻨﺬﺭﯼ ﺍﻟﻤﺘﻮﻓﯽ  ﻫﻖ


In [11]:
def langdetect_custom(text):
    try:
        return detect(text)
    except:
        return None

In [12]:
start = time.time()
langid_df['langdetect_prediction'] = langid_df['Text'].apply(langdetect_custom)
langdetect_time_avg = (time.time() - start) / len(langid_df)

print(f"Accuracy: {accuracy(langid_df, 'language_code', 'langdetect_prediction')}")
print(f"Average time per example: {langdetect_time_avg}")

Accuracy: 0.8434090909090909
Average time per example: 0.003995423707095059


# Exercise 2

For English-written text, apply word-level tokenization. What is the average number of words per sentence?

Implement word-tokenization using both [nltk](https://www.nltk.org/) and [spacy](https://spacy.io/). Report the results for both of them.

For spaCy use the `en_core_web_sm` model.

Let's extract English-written text

In [15]:
en_df = langid_df.loc[langid_df['language_code'] == 'en']
en_df.reset_index(drop=True, inplace=True)
en_df.head()

,Text,language,language_code,fastlangid_prediction,langid_prediction,langdetect_prediction
0,in johnson was awarded an american institute ...,English,en,en,en,en
1,bussy-saint-georges has built its identity on ...,English,en,en,en,en
2,minnesotas state parks are spread across the s...,English,en,en,en,en
3,nordahl road is a station served by north coun...,English,en,en,en,en
4,a talk by takis fotopoulos about the internati...,English,en,en,en,en


Let's install the libraries

In [14]:
!pip3 install nltk
!pip install spacy

Import the libraries

In [24]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

import spacy
spacy_en = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Let's tokenize each sentence with both libraries

In [44]:
en_df.loc[:, 'spacy_tokenizer'] = en_df['Text'].apply(spacy_en)
en_df.loc[:, 'nltk_tokenizer'] = en_df['Text'].apply(word_tokenize)

In [54]:
def average_number_tokens_per_sentence(df, feature):
    return df[feature].apply(len).mean()

In [55]:
print("The average number of words per sentence using nltk:", average_number_tokens_per_sentence(en_df, 'nltk_tokenizer'))
print("The average number of words per sentence using spacy:", average_number_tokens_per_sentence(en_df, 'spacy_tokenizer'))

The average number of words per sentence using nltk: 68.752
The average number of words per sentence using spacy: 72.334


# Exercise 3

Dependency Parsing aims at analyzing the grammatical structure of sentences. The main goal is to find out related words as well as the type of the relationship between them.

The output of this step is a dependency tree similar to the one reported in the figure below.

![dependency tree](http://www.rangakrish.com/wp-content/uploads/2018/04/Deptree-example2.png)

Use spacy to parse the dependency tree of a **randomly selected** sentence. You can both use English sentences or your native language (if supported in [spaCy](https://spacy.io/usage/models/)). Use [displaCy](https://explosion.ai/demos/displacy) to visualize the result in the notebook.

In [58]:
!python -m spacy download es_core_news_sm
spacy_es = spacy.load("es_core_news_sm")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 73.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Now, we choose randomly a sentence in Spanish (My native language which is supported in spaCy)

In [73]:
es_sentence = langid_df.loc[langid_df['language_code'] == 'es', 'Text'].sample().values[0]
es_sentence

'nacido en hartford cheshire inglaterra realizó sus estudios en el ardingly college a los catorce años dejó la escuela para hacerse actor teatral y tres años después viajó a estados unidos para empezar a trabajar en broadway actuando en  en la pieza titulada the first born'

In [79]:
from spacy import displacy
displacy.render(spacy_es(es_sentence), style='dep')

The same exercise but with the given sentence "Vicky is my pet dog"

In [78]:
displacy.render(spacy_en('Vicky is my pet dog'), style='dep')

# Exercise 4
For the same sentence selected in the previous step apply all the following steps:
1. Lemmatization: convert each word to its root form.
2. Stopword removal: remove language-specific stopwords.
3. Part of Speech Tagging: for each word in the sentence display its part-of-speech.

For each step, print the resulting list on the console.
<hr/>
1. Lemmatization

In [123]:
es_sentence_lemmatized = ' '.join([token.lemma_ for token in spacy_es(es_sentence.lower())])
es_sentence_lemmatized

'nacido en hartford cheshire inglaterra realizar su estudio en el ardingly college a el catorce año dejar el escuela para hacer él actor teatral y tres año después viajar a estados unido para empezar a trabajar en broadway actuar en   en el pieza titulado the first born'

2. Stopword removal

In [124]:
from spacy.lang.es.stop_words import STOP_WORDS

In [125]:
es_sentence_lemma_without_sw = ' '.join([token.text.lower() for token in spacy_es(es_sentence_lemmatized) if token.text.lower() not in STOP_WORDS])
es_sentence_lemma_without_sw

'nacido hartford cheshire inglaterra estudio ardingly college catorce año dejar escuela actor teatral año viajar unido empezar trabajar broadway actuar    pieza titulado the first born'

3. Part of Speech Tagging: For each word display its part-of-speech

In [126]:
for t in spacy_es(es_sentence_lemma_without_sw):
    print(f"Token={t} PoS={t.pos_}")

Token=nacido PoS=ADJ
Token=hartford PoS=NOUN
Token=cheshire PoS=NOUN
Token=inglaterra PoS=ADJ
Token=estudio PoS=PROPN
Token=ardingly PoS=PROPN
Token=college PoS=ADJ
Token=catorce PoS=NUM
Token=año PoS=NOUN
Token=dejar PoS=VERB
Token=escuela PoS=PROPN
Token=actor PoS=PROPN
Token=teatral PoS=ADJ
Token=año PoS=NOUN
Token=viajar PoS=VERB
Token=unido PoS=ADJ
Token=empezar PoS=VERB
Token=trabajar PoS=VERB
Token=broadway PoS=NOUN
Token=actuar PoS=VERB
Token=    PoS=SPACE
Token=pieza PoS=NOUN
Token=titulado PoS=ADJ
Token=the PoS=PROPN
Token=first PoS=PROPN
Token=born PoS=PROPN


# **Occurrence-based text representation - TF-IDF**

---

TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. It allows to create occurrence-based vector representation for each document.

# Exercise 5
Use TF-IDF to vectorize each sentence in the original data collection. You can choose your preferred implementation for TF-IDF vectorization. It is also available on [SciKit-Learn library](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
# your code here

# Exercise 6

Build a supervised multi-class language detector using as features the vector obtained by TF-IDF representation. Use 80% of the data to train the language detector and 20% of the data for assessing its accuracy.

In [ ]:
# your code here

# **Topic Modeling**
---
Occurrence-based representations are high-dimensional, what is the dimension of the generated TF-IDF vector representation?
Topic modeling focuses on capturing latent topics in large document corpora.

The data collection used in this second part of the practice is provided [here](https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/CovidFake_filtered.csv) - [source: Zenodo](https://zenodo.org/record/4282522#.YVdCXcbOOpd)


# Exercise 7

Latent Semantic Indexing (LSI) models underlying concepts by using SVD (Singular Value Decomposition).

Use [gensim](https://radimrehurek.com/gensim/) library to:
1. Create a corpus composed of the headlines contained in the data collection.
2. Generate a [dictionary](https://radimrehurek.com/gensim/corpora/dictionary.html) to create a word -> id mapping (required by LSI module).
3. Using the dictionary, preprocess the corpus to obtain the representation required for LSI model training ([documentation here](https://radimrehurek.com/gensim/models/lsimodel.html)).
4. Inspect the top-5 topics generated by the LSI model for the analysed corpus.

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/CovidFake_filtered.csv

In [ ]:
# your code here

# Exercise 8

The top-scored words contributing to each topic (if no stopword removal is applied) are english common words (e.g., *to, for, in, of, on*..). Moreover, missing punctuation removal could be critical for topic identification. Repeat the same procedure of Ex. 7 by adding preliminary preprocessing step to:
1. **remove stopwords**
2. **strip punctuation**
3. **lowercase all words**

In [ ]:
# your code here

# Exercise 9

Leveraging the same corpus used for LSI model generation, apply LDA modeling setting the number of topics to 5. Display the words most contributing to the those topics according to the LDA model.

In [ ]:
# your code here

# Exercise 10

Using [pyLDAvis]() library build an interactive visualization for the trained LDA model.

In [ ]:
# your code here

# Exercise 11
**Credits:** Giuseppe Gallipoli

#### Introduction
[Large Language Models](https://en.wikipedia.org/wiki/Large_language_model) (LLMs) are a type of deep learning model capable of language generation. These models are built on deep learning architectures, primarily using neural networks, and are trained on massive amounts of text data. LLMs generally leverage the *Transformer* architecture (a groundbreaking deep architecture that you will study in more detail later in the course), which allows them to process language in context, capturing complex relationships between words and concepts.

Large Language Models have demonstrated excellent capabilities across a wide variety of tasks, making them versatile models which can be applied in diverse scenarios and use cases.

Given their relevance, although you have not yet covered this topic in the course, we will provide you, starting from this first laboratory practice, with practical applications showing how LLMs can be used to solve a diverse range of tasks.\
Don't worry about the theoretical or more technical aspects: they will be covered in more detail in due time.\
For now, the most important thing to know is that users interact with LLMs by means of a **prompt**, which is a piece of text containing the instruction or question the user wants to give or ask the model.

#### Topic Modeling using Large Language Models

In this practice, we will use a Large Language Model to address a topic modeling-related task. Specifically, rather than modeling topic distributions as done with techniques like LSI or LDA, we will ask the LLM to extract the most relevant topic(s) from sentences (or from an entire corpus) according to different approaches.

For this task, we will use the [Zephyr](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta) 7B model, i.e., `HuggingFaceH4/zephyr-7b-beta`.

**1<sup>st</sup> approach**: Ask the model to identify the topic(s) contained in a given sentence <u>without providing</u> a predefined list of topics to choose from.\
*Example of prompt*:\
Which are the most relevant topics of the following sentence?

\
<u>Suggestion</u>: To increase speed, switch to a GPU runtime. You can do this by clicking on Runtime → Change runtime type → Hardware accelerator → Select T4 GPU.\
If you encounter an `OutOfMemoryError`, try restarting the session by clicking on Runtime → Restart session.

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/CovidFake_filtered.csv

In [ ]:
# fill in the following code

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd, random

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

df_tmodeling = ...

PROMPT = "Write your prompt here"

# It may take some time to download the model
model = AutoModelForCausalLM.from_pretrained('HuggingFaceH4/zephyr-7b-beta', torch_dtype=torch.float16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained('HuggingFaceH4/zephyr-7b-beta')

for ...
  full_prompt = ...
  input = tokenizer(full_prompt, return_tensors='pt').to(device)
  output = model.generate(**input, max_new_tokens=32)
  output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  ...

**2<sup>nd</sup> approach**: Ask the model to identify the topic(s) contained in a given sentence <u>providing</u> a predefined list of topics to choose from.

*Example of prompt*:\
Which are the most relevant topics of the following sentence?\
Choose among: medicine, COVID, Artificial Intelligence, treatment, English literature, vaccine, gardening

In [ ]:
# fill in the following code

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd, random

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

df_tmodeling = ...

PROMPT = "Write your prompt here"

# It may take some time to download the model. If you have already downloaded and loaded it, you can skip this part
model = AutoModelForCausalLM.from_pretrained('HuggingFaceH4/zephyr-7b-beta', torch_dtype=torch.float16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained('HuggingFaceH4/zephyr-7b-beta')

for ...
  full_prompt = ...
  input = tokenizer(full_prompt, return_tensors='pt').to(device)
  output = model.generate(**input, max_new_tokens=32)
  output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  ...

**3<sup>rd</sup> approach**: Ask the model to identify the topic(s) contained in a given sentence <u>providing</u> a predefined list of topics to choose from along with the corresponding definitions.

*Example of prompt*:\
Which are the most relevant topics of the following sentence?\
Choose among:
- medicine: treatment for illness or injury, or the study of this
- COVID: an infectious disease caused by a coronavirus
- Artificial Intelligence: computer systems that have some of the qualities that the human brain has, such as learn from data
- treatment: the use of drugs to cure a person of an illness or injury
- English literature: artistic works written in the English language, especially those with a high and lasting artistic value
- vaccine: a substance that is put into the body of a person or animal to protect them from a disease
- gardening: the job or activity of working in a garden

Definitions are taken from the [Cambridge Dictionary](https://dictionary.cambridge.org/) and have been slightly adapted.

In [ ]:
# fill in the following code

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd, random

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

df_tmodeling = ...

PROMPT = "Write your prompt here"

# It may take some time to download the model. If you have already downloaded and loaded it, you can skip this part
model = AutoModelForCausalLM.from_pretrained('HuggingFaceH4/zephyr-7b-beta', torch_dtype=torch.float16, device_map=device)
tokenizer = AutoTokenizer.from_pretrained('HuggingFaceH4/zephyr-7b-beta')

for ...
  full_prompt = ...
  input = tokenizer(full_prompt, return_tensors='pt').to(device)
  output = model.generate(**input, max_new_tokens=32)
  output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
  ...

After manually inspecting some of the outputs for each approach, here you can find some questions to reason about the results:
- Did you find an approach which worked best overall?
- Did you encounter any cases where the LLM failed?
- What happens if all the topics provided are irrelevant to the sentence?
- Does the presence of definitions improve the model's performance?
- What challenges or limitations did you observe?